In [ ]:
# prompt: check if we are using a gpu

import tensorflow as tf

if not tf.config.list_physical_devices('GPU'):
  print("No GPU available")


## Get data

In [ ]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git
!ls pubmed-rct

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 39 (delta 8), reused 5 (delta 5), pack-reused 25
Receiving objects: 100% (39/39), 177.08 MiB | 26.45 MiB/s, done.
Resolving deltas: 100% (15/15), done.
PubMed_200k_RCT				       PubMed_20k_RCT_numbers_replaced_with_at_sign
PubMed_200k_RCT_numbers_replaced_with_at_sign  README.md
PubMed_20k_RCT


In [ ]:
# prompt: check what files are in the pubmed_20k

!ls /content/pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign


dev.txt  test.txt  train.zip


In [6]:
data_dir = '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign'

In [39]:
train_dir = '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt'
test_dir = '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt'
val_dir = '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt'



In [11]:
# prompt: check all the filenames in the data_dir

tf.io.gfile.listdir(data_dir)


['train.txt', 'dev.txt', 'test.txt']

## Preprocess data
Time to become one with the data

In [16]:
# prompt: create function to read the lines of a document

def read_lines(path):
  with open(path, 'r') as f:
    lines = f.readlines()
  return lines

train_lines = read_lines(train_dir)
train_lines[:20]

['###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'METHODS\tA total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .\n',
 'METHODS\tOutcome measures included pain reduction and improvement in function scores and systemic inflammation markers .\n',
 'METHODS\tPain was assessed using the visual analog pain scale ( @-@ mm ) .\n',
 'METHODS\tSecondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and @-min walk distance ( @MWD ) .\n',
 'METHODS\tSerum levels of interleukin @ ( IL-@ ) , IL-@ , tumor necrosis factor ( TNF ) - , and 

In [68]:
# Initialize variables
result_list = []
line_number = 0
doc_id = None

# Iterate through the data
for line in train_lines:
    line_number += 1

    # Split the line into parts based on tabs
    parts = line.strip().split('\t')

    # Check if the line has the expected format
    if len(parts) < 2:
        # Skip lines that don't have enough parts
        continue

    # Extract information
    if parts[0].startswith('###'):
        # This line contains the document ID
        doc_id = parts[0][3:]
    else:
        # Extract target and text
        target = parts[0]
        text = parts[1]


        # Create a dictionary for the current entry
        entry_dict = {'doc_id': doc_id, 'target': target, 'text': text, 'total_lines': len(parts)}

        # Append the dictionary to the list
        result_list.append(entry_dict)

# Print the result list
# for entry in result_list:
#     print(entry)


In [61]:
len(result_list)

180040

In [46]:
def preprocess_text_with_line_numbers(filename):
  """Returns a list of dictionaries of abstract line data.

  Takes in filename, reads its contents and sorts through each line,
  extracting things like the target label, the text of the sentence,
  how many sentences are in the current abstract and what sentence number
  the target line is.

  Args:
      filename: a string of the target text file to read and extract line data
      from.

  Returns:
      A list of dictionaries each containing a line from an abstract,
      the lines label, the lines position in the abstract and the total number
      of lines in the abstract where the line is from. For example:

      [{"target": 'CONCLUSION',
        "text": The study couldn't have gone better, turns out people are kinder than you think",
        "line_number": 8,
        "total_lines": 8}]
  """
  input_lines = read_lines(filename) # get all lines from filename
  abstract_lines = "" # create an empty abstract
  abstract_samples = [] # create an empty list of abstracts

  # Loop through each line in target file
  for line in input_lines:
    if line.startswith("###"): # check to see if line is an ID line
      abstract_id = line
      abstract_lines = "" # reset abstract string
    elif line.isspace(): # check to see if line is a new line
      abstract_line_split = abstract_lines.splitlines() # split abstract into separate lines

      # Iterate through each line in abstract and count them at the same time
      for abstract_line_number, abstract_line in enumerate(abstract_line_split):
        line_data = {} # create empty dict to store data from line
        target_text_split = abstract_line.split("\t") # split target label from text
        line_data["target"] = target_text_split[0] # get target label
        line_data["text"] = target_text_split[1].lower() # get target text and lower it
        line_data["line_number"] = abstract_line_number # what number line does the line appear in the abstract?
        line_data["total_lines"] = len(abstract_line_split) - 1 # how many total lines are in the abstract? (start from 0)
        abstract_samples.append(line_data) # add line data to abstract samples list

    else: # if the above conditions aren't fulfilled, the line contains a labelled sentence
      abstract_lines += line

  return abstract_samples

train_samples = preprocess_text_with_line_numbers(train_dir)
test_samples = preprocess_text_with_line_numbers(test_dir)
val_samples = preprocess_text_with_line_numbers(val_dir)

In [47]:
len(train_samples), len(test_samples), len(val_samples)

(180040, 30135, 30212)

In [69]:
train_samples[:10]

[{'target': 'OBJECTIVE',
  'text': 'to investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( oa ) .',
  'line_number': 0,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'a total of @ patients with primary knee oa were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .',
  'line_number': 1,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'outcome measures included pain reduction and improvement in function scores and systemic inflammation markers .',
  'line_number': 2,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'pain was assessed using the visual analog pain scale ( @-@ mm ) .',
  'line_number': 3,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'secondary outcome measures included the western ontari